In [3]:
%%html
<style>
table {float:left}
</style>

|Meaning|EBNF Grammer|Regular Expression|
|-------|------------|------------------|
|repetition|{}|*|
|optional|[]|?|
|example 1|['a']|(ab)?|
|example 2|{'ab'}|(ab)*|

    p ::= a {',' a}
    a ::= 'e' | '[' p ']' | '(' p ')'

Space doesn't matter in EBNF.

For an EBNF grammar, the two conditions for recursive descent parsing can be phrased as follows:

| `E`           | `condition(E)`                               |
|:--------------|:---------------------------------------------|
| `[E₁]`        | `first(E₁) ∩ follow(E) = {}`                 |
| `{E₁}`        | `first(E₁) ∩ follow(E) = {}`                 |
| `E₁ E₂ …`     | `first(Eᵢ) ∩ first(Eᵢ₊₁ Eᵢ₊₂ …) = {}` for any nullable `Eᵢ`, provided `E` is not nullable |
| `E₁ │ E₂ │ …` | `first(Eᵢ) ∩ first(Eⱼ) = {}` for all `i ≠ j` |

`p ::= a {',' a}` need to apply `E1E2` and `{E1}`

For the first production, both conditions hold. Empty set means disjoint.

    E₁ E₂ … → apply only when either E₁, E₂ or Eᵢ is nullable, which is not.
    E₁ E₂ … → first(a) ∩ first({',' a})
              = {'e', '[', '('} ∩ {','}
              = {}
    {E₁} → first({',' a}) ∩ follow(p)
           = {','} ∩ {']', ')'}
           = {}

For the second production, the conditions hold

    E₁ │ E₂ │ … → first('e') ∩ first('[' p ']') = {}
                → first('e') ∩ first('(' p ')') = {}
                → first('[' p ']') ∩ first('(' p ')') = {}

Create the parser in Python

    p ::= a {',' a}
    a ::= 'e' | '[' p ']' | '(' p ')'

In [33]:
def getCh():
    global ch, source
    if len(source) > 0: ch, source = source[0], source[1:]
    else: ch = None 

def depth(s):
    global source
    source = s; getCh(); p()
    if ch != None: raise 

def a():
    if ch == 'e': getCh()
    elif ch == '[':
        getCh(); p()
        if ch == ']': getCh()
        else: raise
    elif ch == '(':
        getCh(); p()
        if ch == ')': getCh()
        else: raise
    else: raise
def p():
    a()
    while ch == ',': getCh(); a()

The depth of an expression is the maximal level of nested parenthesis or brackets. Extend the grammar with attribute rules that compute the depth. Apply attribute rules by «  ».